In [1]:
import pandas as pd 
import numpy as np 
import pickle

In [78]:
from sklearn.svm import SVC, OneClassSVM
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier as RFC, IsolationForest as IF

In [10]:
## Loading Model Results
data_dir = "/media/gvk/GVK_Elements/github_repo/Ensemble/Stacking_Test/Stacking_Data/Ensemble_pcd885.csv"
results_df= pd.read_csv(data_dir)

In [11]:
results_df

,fname,color_iou,color_quality,depth_iou,depth_quality,ggcnn_iou,ggcnn_quality,ggcnn2_iou,ggcnn2_quality,ground_truth
0,pcd0100,1,1.051706,1,0.954668,1,0.789719,1,0.954668,1
1,pcd0101,1,1.045923,1,0.953750,1,0.898149,1,0.953750,1
2,pcd0102,1,0.929282,1,0.857348,1,0.770612,1,0.857348,1
3,pcd0103,1,0.993950,1,0.851716,1,0.901683,1,0.851716,1
4,pcd0104,1,0.945183,1,0.825188,1,0.790561,1,0.825188,1
...,...,...,...,...,...,...,...,...,...,...
880,pcd1030,1,0.802078,1,0.495141,0,0.923583,1,0.495141,1
881,pcd1031,0,0.663789,0,0.968811,1,0.877918,0,0.968811,1
882,pcd1032,0,0.848007,1,1.020501,1,1.065817,1,1.020501,1
883,pcd1033,1,0.990940,1,0.660984,1,0.754636,1,0.660984,1


In [13]:
X = results_df.drop(columns= ['fname', 'ground_truth']).to_numpy()
y = results_df['ground_truth'].to_numpy()

In [32]:
lenX = len(X)
ratio = int(0.9 * lenX)
inds = np.random.permutation(np.arange(lenX))
trainX, testX = X[:ratio], X[ratio:]
trainY, testY = y[:ratio], y[ratio:]

In [66]:
color_trainY, color_testY = X[:ratio, 0], X[ratio:, 0]
depth_trainY, depth_testY = X[:ratio, 2], X[ratio:, 2]
ggcnn_trainY,  ggcnn_testY= X[:ratio, 4], X[ratio:, 4]
ggcnn2_trainY, ggcnn2_testY = X[:ratio, 6], X[ratio:, 6]


In [71]:
color_predict = X[:, 0]
depth_predict = X[:, 2]
ggcnn_predict = X[:, 4]
ggcnn2_predict = X[:, 6]

In [86]:
results_SVC = SVC(random_state = 0,  probability = True)
results_SVC.fit(X = trainX, y = trainY)

ValueError: The number of classes has to be greater than one; got 1 class

In [84]:
## Once Class SVM
results_OCSVC = OneClassSVM()
results_OCSVC.fit(X = trainX, y = trainY)
predictY_OCSVC = results_OCSVC.predict(X = testX)
print(classification_report(testY, predictY_OCSVC))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         0
           1       1.00      0.47      0.64        89

    accuracy                           0.47        89
   macro avg       0.50      0.24      0.32        89
weighted avg       1.00      0.47      0.64        89

/media/gvk/GVK_Elements/py_linux/ensemble_dr_linux/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/gvk/GVK_Elements/py_linux/ensemble_dr_linux/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, 

In [85]:
## Random Forest
results_RFC = RFC(random_state = 0)
results_RFC.fit(X = trainX, y = trainY)
predictY_RFC = results_RFC.predict(X = testX)
print(classification_report(testY, predictY_RFC))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00        89

    accuracy                           1.00        89
   macro avg       1.00      1.00      1.00        89
weighted avg       1.00      1.00      1.00        89



In [83]:
## Isolation Forest
results_IF = IF(random_state = 0)
results_IF.fit(X = trainX, y = trainY)
predictY_IF = results_IF.predict(X = testX)
print(classification_report(testY, predictY_IF))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         0
           1       1.00      0.58      0.74        89

    accuracy                           0.58        89
   macro avg       0.50      0.29      0.37        89
weighted avg       1.00      0.58      0.74        89

/media/gvk/GVK_Elements/py_linux/ensemble_dr_linux/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/gvk/GVK_Elements/py_linux/ensemble_dr_linux/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, 

In [73]:
## For Fadi Color Images
print(classification_report(y, color_predict))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.86      0.92       885

    accuracy                           0.86       885
   macro avg       0.50      0.43      0.46       885
weighted avg       1.00      0.86      0.92       885

/media/gvk/GVK_Elements/py_linux/ensemble_dr_linux/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/gvk/GVK_Elements/py_linux/ensemble_dr_linux/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, 

In [74]:
## For Fadi Depth Images
print(classification_report(y, depth_predict))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.76      0.86       885

    accuracy                           0.76       885
   macro avg       0.50      0.38      0.43       885
weighted avg       1.00      0.76      0.86       885

/media/gvk/GVK_Elements/py_linux/ensemble_dr_linux/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/gvk/GVK_Elements/py_linux/ensemble_dr_linux/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, 

In [75]:
## For GGCNN Depth Images
print(classification_report(y, ggcnn_predict))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.66      0.80       885

    accuracy                           0.66       885
   macro avg       0.50      0.33      0.40       885
weighted avg       1.00      0.66      0.80       885

/media/gvk/GVK_Elements/py_linux/ensemble_dr_linux/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/gvk/GVK_Elements/py_linux/ensemble_dr_linux/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, 

In [76]:
## For GGCNN2 Depth Images
print(classification_report(y, ggcnn2_predict))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.76      0.86       885

    accuracy                           0.76       885
   macro avg       0.50      0.38      0.43       885
weighted avg       1.00      0.76      0.86       885

/media/gvk/GVK_Elements/py_linux/ensemble_dr_linux/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/gvk/GVK_Elements/py_linux/ensemble_dr_linux/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, 